In [40]:
from bs4 import BeautifulSoup
import lxml
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.support import expected_conditions as EC
import csv
import re

chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument("--blink-settings=imagesEnabled=false") # disabling images to improve run time 
#chrome_options.add_argument('--headless')  # Enable headless mode
#chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration in headless mode (optional)
driver = webdriver.Chrome(options=chrome_options)

url = "https://letterboxd.com/film/parasite-2019/reviews/by/activity/page/1/"

source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
divs = soup.find_all("div", class_ = 'body-text -prose collapsible-text') # these contain the reviews

count = 1

for i in divs: #iterate through reviews of first page,sorted by activit, 12 for each page 
    
    review = i.p
    if review.text == "This review may contain spoilers. I can handle the truth.": # skipping reviews with spoilers
        # this can easily be found, the paths to it are kinda confusing tho 
        # you have to go back into the original div without .p and find the hidden tag
        review = i.find("div", class_ = "hidden-spoilers expanded-text")
        
    if review.text.find("…") != -1: # skipping reviews that are too long 
# note: i notice that reviews that are too long have this unique ... thing that you can copy and paste
# it detects it easily so we can weed it out but i believe we will have to use selenium for this part 
# hopefully we will only use selenium for this part as to minimize run time 
# we should use find by CSS selector for this to minimize time hopefully, 
# shouldn't be too hard u just have to look up format
        driver.get(url)
        
        more_button = driver.find_element(By.XPATH, f"//*[@id='content']/div/div/section/section/ul/li[5]/div/div[2]/div/p/a")
        driver.execute_script("arguments[0].click();", more_button)
        
        #wait=WebDriverWait(driver,20)
        #more_button= wait.until(EC.element_to_be_clickable((By.XPATH, f"//*[@id='content']/div/div/section/section/ul/li[{count}]/div/div[2]/div/p/a")))
        #more_button.click()
        
        #WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,f"//*[@id='content']/ul/li[{count}]/p/a"))).click()
        #more_button.click()
        #review = driver.find_element(By.CSS_SELECTOR, f"#content ul li:nth-child({count}) .body-text.-prose.collapsible-text")
    driver.close()
    review = review.text.strip()
    count += 1
    print(review)
# I am thinking that for implementation, we can do a similar for loop like this. we just need to put selenium 
# maybe write as we go? not sure how we want to do the csv part 
# we also need to find the best way to attatch the title and year to it 
#i'm thinking the obvious way is to match the links? however we must remove the review portion of the link 
# now that i think of it i think we need a new csv for sure, with a column for each movie or each row?
# probably each row if that's easier to write... now how many reviews do we want?
# because we have so many movies, gathering reviews will be x times time consuming 
# we also need to consider that less popular movies will have less pages... probably needs a try except bit 
# do we want to measure reviews in proportion to popularity?
# i mean, for parasite there are hundreds of pages, for terror 2000, there are not even 50 
# also these reviews are funny
# much to think about... dot dot dot 



Update: Now in video formThere is a house on a hill, and there are people in it. And they're nice, but not really. There is a garden, and there they get a lot of sunlight. And it is nice, and they get to sleep there sometimes.There is a house underground, and there are people in it. And they're the first to tell you that they're not very nice: they're con-men, grifters, forgers and imposters. They sit among stink…


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000104697f48 chromedriver + 4226888
1   chromedriver                        0x00000001046904f4 chromedriver + 4195572
2   chromedriver                        0x00000001042d4be8 chromedriver + 281576
3   chromedriver                        0x0000000104303b70 chromedriver + 473968
4   chromedriver                        0x0000000104304ef4 chromedriver + 478964
5   chromedriver                        0x000000010465959c chromedriver + 3970460
6   chromedriver                        0x000000010465d6f0 chromedriver + 3987184
7   chromedriver                        0x00000001046635b4 chromedriver + 4011444
8   chromedriver                        0x000000010465e2fc chromedriver + 3990268
9   chromedriver                        0x00000001046361c0 chromedriver + 3826112
10  chromedriver                        0x000000010467a088 chromedriver + 4104328
11  chromedriver                  

In [71]:
from bs4 import BeautifulSoup
import lxml
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import csv
import re

chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument("--blink-settings=imagesEnabled=false") # disabling images to improve run time 
chrome_options.add_argument('--headless')  # Enable headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration in headless mode (optional)
driver = webdriver.Chrome(options=chrome_options)
action = ActionChains(driver)

url = "https://letterboxd.com/film/parasite-2019/reviews/by/activity/page/1/"

driver.get(url)
wait=WebDriverWait(driver,5)
more_button= wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='content']/div/div/section/section/ul/li[5]/div/div[2]/div/p/a")))

#ActionChains(driver).move_to_element(wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='content']/div/div/section/section/ul/li[5]/div/div[2]")))).perform()
ActionChains(driver).move_to_element(more_button).click(more_button).perform()
review = driver.find_element(By.XPATH, "//*[@id='content']/div/div/section/section/ul/li[5]/div/div[2]/div/p").text
#more_button = driver.find_element(By.XPATH, "//*[@id='content']/div/div/section/section/ul/li[5]/div/div[2]")
#driver.execute_script("arguments[0].click();", more_button)
#wait=WebDriverWait(driver,5)
#more_button= wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='content']/div/div/section/section/ul/li[5]/div/div[2]/div")))

#more_button.click()

#review = driver.find_element(By.CSS_SELECTOR, f"#content ul li:nth-child(5) .body-text.-prose.collapsible-text").text


driver.close()
print(review)

One detail I noticed this time around is that Min and Mr. Park both react the same way to being asked if they like or love the people that they're romantically involved with– by laughing. And in Mr. Park's case, giving a sarcastic answer that undercuts his saying 'yes'. Min and Mr. Park are both seen as powerful figures deserving of respect, and the way they dismissively respond to an earnest question about whether they truly care for the people… more
